## Assignment 2
### Collaborators: Aldrin Ilagan, Lindsey Rappaport, Yvanna Cardenas

In [1]:
# Import packages (same from HW3)

import random
from glob import glob
from collections import defaultdict

import numpy as np
from numpy.random import choice

from symusic import Score
from miditok import REMI, TokenizerConfig
from midiutil import MIDIFile

In [2]:
random.seed(42)

### Symbolic,  unconditioned generation